In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper


In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv= ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name 

'arxiv'

In [4]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name 

'wikipedia'

In [5]:
tools=[wiki, arxiv]

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
google_api_key = os.getenv("GENAI_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=google_api_key)
vector = embeddings.embed_query("hello, world!")
vector[:5]



[-0.02276923693716526,
 0.010134130716323853,
 0.011886735446751118,
 -0.09669032692909241,
 -0.0027089761570096016]

In [23]:
loader = WebBaseLoader("https://huggingface.co/docs/inference-providers/index")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vector_storedb = FAISS.from_documents(documents, embeddings)
retriever = vector_storedb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A0F2BA37D0>, search_kwargs={})

In [26]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "hugging_face-search","Search any information about Why Choose Inference Providers")
retriever_tool.name

'hugging_face-search'

In [28]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\Admin\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='hugging_face-search', description='Search any information about Why Choose Inference Providers', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001A0FCAA5080>, retriever=VectorStoreRetriever(tags=['FAI

In [29]:
from dotenv import load_dotenv
load_dotenv()
import os

groq_api_key = os.getenv("GROQ_API_KEY")


from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = groq_api_key, model="llama-3.1-8b-instant")


In [30]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [31]:
from langchain.agents import create_openai_tools_agent 
agents = create_openai_tools_agent(llm, tools, prompt)
agents

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [32]:
##agents Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agents, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [34]:
agent_executor.invoke({
	"input": "Tell me about hugging face inference. Only use the available tools: wikipedia, arxiv, or hugging_face-search. Do not use any other tools."
})



> Entering new AgentExecutor chain...

Invoking: `hugging_face-search` with `{'query': 'Inference in hugging face'}`


hf auth login # get a read token from hf.co/settings/tokens You can now use the the client with a Python interpreter:   Copied import os
from huggingface_hub import InferenceClient

Inference Providers   Hugging Face’s Inference Providers give developers access to hundreds of machine learning models, powered by world-class inference providers. They are also integrated into our client SDKs (for JS and Python), making it easy to explore serverless inference of models on your favorite providers.  Partners Our platform integrates with leading AI infrastructure providers, giving you access to their specialized capabilities through a single, consistent API. Here’s what each partner supports: Provider Chat completion (LLM) Chat completion (VLM) Feature Extraction Text to Image Text to video Speech to text Cerebras ✅      Cohere ✅ ✅     Fal AI    ✅ ✅ ✅ Featherless AI ✅ ✅    

{'input': 'Tell me about hugging face inference. Only use the available tools: wikipedia, arxiv, or hugging_face-search. Do not use any other tools.',
 'output': 'Note: The second arxiv result was cut off due to it being too long.'}

In [35]:
agent_executor.invoke({
	"input": "what is machine learning"
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `hugging_face-search` with `{'query': 'Why Choose Inference Providers'}`


✅ ✅   ✅  Nscale ✅ ✅  ✅   Public AI ✅      Replicate    ✅ ✅ ✅ SambaNova ✅  ✅    Scaleway ✅  ✅    Together ✅ ✅  ✅    Why Choose Inference Providers? When you build AI applications, it’s tough to manage multiple provider APIs, comparing model performance, a

{'input': 'what is machine learning',
 'output': 'The function call for the summary of the \'arxiv\' function call was incomplete. This is the complete function call for the \'arxiv\' function:\n\n <function=arxiv>{"query": "machine learning"}</function>\n\nIf you want to know more about the results, you can ask me to clarify any part of the response.'}